<a href="https://colab.research.google.com/github/arnavm30/EmotionRecognition/blob/main/Public_Emotion_Recognition_Facemesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and importing modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install mediapipe pandas numpy opencv-python imbalanced-learn keras keras-tuner

In [ ]:
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
import os
import csv

# Creating CSV of landmarks from images in ExpW

Only run the first time: csv saved to drive after.

Generating header row

In [ ]:
num_coords = 468
header_row = ['expression_label']

for i in range(1, num_coords+1):
    header_row += [f'x{i}', f'y{i}', f'z{i}']

with open('/home/arnav/Workarea/MentalStateProject/facemesh_landmarks.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(header_row)

Helper function that maps image to emotion label

In [ ]:
def lookup(image_name):
    with open('/home/arnav/Workarea/MentalStateProject/label.lst') as f:
        for line in f:
            if image_name in line:
                return line[-2] 

Generating face mesh for each static image with one face (with multiple returns None) and then adding x, y, z of all 468 landmarks to each column of csv. Also padding and resizing image to (224,224) and organizing images by label into corresponding folder.

In [ ]:
mp_holistic = mp.solutions.holistic
images_dir = '/home/arnav/Workarea/MentalStateProject/origin'
images_dir_new = '/home/arnav/Workarea/MentalStateProject/origin2'
i = 0
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for image_name in os.listdir(images_dir):
        i += 1
        if image_name.endswith(".jpg"):
            expression_label = lookup(image_name)
            image = cv2.imread(f'{images_dir}/{image_name}')
            h, w, c = image.shape
            if h > w:
                image = cv2.copyMakeBorder(image, 0, 0, (h-w)//2, (h-w)//2 + 1, cv2.BORDER_CONSTANT, value=[255,255,255])
            else:
                image = cv2.copyMakeBorder(image, (w-h)//2, (w-h)//2 + 1, 0, 0, cv2.BORDER_CONSTANT, value=[255,255,255])
            image = cv2.resize(image, (224, 224))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            landmarks = holistic.process(image)
            print(i)
            print(image_name)
            print(landmarks.face_landmarks)
            if landmarks.face_landmarks is not None: # multiple faces returns None
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                cv2.imwrite(f'{images_dir_new}/{expression_label}/{image_name}', image)
                
                facemesh = landmarks.face_landmarks.landmark # (x, y, z)

                face_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in facemesh]).flatten())
                face_row.insert(0, image_name)
                face_row.insert(0, expression_label)

                with open ('/home/arnav/Workarea/MentalStateProject/facemesh_landmarks2.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(face_row)

# Preproccessing

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/facemesh_landmarks.csv')
filtered_df = df[df['expression_label'].notnull()]
df = filtered_df

for expression_label：

"0" "angry"

"1" "disgust"

"2" "fear"

"3" "happy"

"4" "sad"

"5" "surprise"

"6" "neutral"


In [ ]:
X = df.drop('expression_label', axis=1) # features
y = df['expression_label'] # target

One hot encoding the class column

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y = lb.fit_transform(y)

Data split pseudo-randomly (with seed) for 67% train, 33% test; stratified so same proportion of classes in train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)

Using RandomUnderSampler, a sampling method to mitigate class imbalance (far more facemeshes for happy and sad compared to other classes)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=13)
X_res, y_res = rus.fit_sample(X_train, y_train)

# ML Models

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import InputLayer, Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.metrics import Precision, Recall

model = Sequential()
model.add(InputLayer(input_shape=X_res.shape[1]))
model.add(Dense(y_res.shape[1]))
model.add(Activation('softmax'))

optimizer = keras.optimizers.Adam(lr=1e-3) # keras default: lr=0.001
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1000, validation_data = (X_test,y_test), batch_size=64)

In [ ]:
model.save('simple_mlp.h5')

In [ ]:
from keras.models import load_model
simple_mlp = load_model('/content/drive/MyDrive/simple_mlp.h5')
simple_mlp.summary()

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=128)
test_loss, test_acc

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
y_hat=np.argmax(y_hat, axis=1)
y_test=np.argmax(y_test, axis=1) 
cm = confusion_matrix(y_test, y_hat)

In [ ]:
print(cm)

In [ ]:
import tensorflow as tf
#from tf.losses import softmax_cross_entropy
from tensorflow import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import Adam
import keras_tuner as kt

model2 = Sequential()
model2.add(Dense(2048, input_dim=X_train.shape[1]))
model2.add(Activation('relu'))
model2.add(Dropout(0.1))
model2.add(Dense(1024))
model2.add(Activation('relu'))
model2.add(Dropout(0.1))
model2.add(Dense(y_train.shape[1]))
model2.add(Activation('softmax'))

epochs = 10000
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam() # keras default: lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0
#opt = tf.train.AdamOptimizer()
model2.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer, 
    metrics=['acc'])

print("Training...")
model2.fit(X_train, y_train, steps_per_epoch=100, epochs=epochs, validation_data = (X_test,y_test), batch_size=64)

model2.save('/content/drive/MyDrive/mlp.h5')

In [ ]:
y_hat = model2.predict(X_test)

In [ ]:
from keras.models import load_model
model2 = load_model('/content/drive/MyDrive/mlp.h5')

In [ ]:
test_loss, test_acc = model2.evaluate(X_test, y_test, batch_size=128)
test_loss, test_acc

In [ ]:
#from tf.losses import softmax_cross_entropy
from tensorflow import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import InputLayer, Dense, Activation, Dropout
from keras.optimizers import Adam
import kerastuner as kt

def model_builder(hp):
  model = Sequential()
  model.add(InputLayer(input_shape=X_res.shape[1]))

  model.add(Activation('relu'))
  #model.add(Dropout(0.15))
  model.add(Dense(512))
  model.add(Activation('relu'))
  #model.add(Dropout(0.15))
  model.add(Dense(y_res.shape[1]))
  model.add(Activation('softmax'))

  lr_schedule = keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=1e-2,
      decay_steps=10000,
      decay_rate=0.9)
  optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  return model

print("Training...")
model2.fit(X_res, y_res, epochs=100, validation_data = (X_test,y_test), batch_size=64)

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, matthews_corrcoef, precision_score, recall_score, f1_score
def compute_metrics(y_test, y_pred):
    print('Accuracy: {:.5f}'.format(accuracy_score(y_test, y_pred)))
    print('F-score: {:.5f}'.format(f1_score(y_test, y_pred)))
    print('Precision: {:.5f}'.format(precision_score(y_test, y_pred)))
    print('Recall: {:.5f}'.format(recall_score(y_test, y_pred)))
    print('Accuracy (balanced): {:.5f}'.format(balanced_accuracy_score(y_test, y_pred)))
    print('MCC: {:.5f}'.format(matthews_corrcoef(y_test, y_pred)))

def compute_confusion_matrix(y_test, y_pred):
    return pd.DataFrame(
        confusion_matrix(y_test, y_pred, labels=[6, 5, 4, 3, 2, 1, 0]),
        columns=['a(x) = 6','a(x) = 5','a(x) = 4','a(x) = 3','a(x) = 2','a(x) = 1', 'a(x) = 0'],
        index=['y = 6', 'y = 5', 'y = 4', 'y = 3', 'y = 2', 'y = 1', 'y = 0'],
    ).T

In [ ]:
compute_confusion_matrix(y_test, y_hat)

# Resnet to extract features concatenated to the facemesh

In [ ]:
!unzip '/content/drive/MyDrive/FaceMesh/origin3.zip'

In [ ]:
import tensorflow
from tensorflow import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import decode_predictions
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from pathlib import *


In [ ]:
resnet = ResNet50(weights='imagenet')

In [ ]:
# load model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3), pooling='max')
# remove the output layer
#model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [ ]:
model.summary()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/FaceMesh/facemesh_landmarks3.csv')

In [ ]:
columns_to_add = []
for i in range(features.shape[1]):
  columns_to_add.append(f'feature{i}')

In [ ]:
df[columns_to_add] = None

In [ ]:
num_coords = 468
num_features = 2048
header_row = ['expression_label', 'image_name']

for i in range(1, num_coords+1):
  header_row += [f'x{i}', f'y{i}', f'z{i}']

for i in range(1, num_features+1):
  header_row += [f'feature{i}']

with open('/content/drive/MyDrive/FaceMesh shared with Mohammad/facemesh_and_features.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(header_row)

In [ ]:
for index, row in df.iterrows():
  image_name = row['image_name']
  expression_label = row['expression_label']
  image = load_img(f'/content/origin3/{expression_label}/{image_name}', target_size=(224, 224))
  # convert the image pixels to a numpy array
  image = img_to_array(image)
  # reshape data for the model
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  # prepare the image for the VGG model
  image = preprocess_input(image)
  # get extracted features
  features = model.predict(image)
  #feature = face_mesh + embedding
  #output_csv.write(f'{feature}\n')

  facemesh = row.T

  #features = np.reshape(features, (2048,))
  #print(features.shape)
  row_to_add = np.concatenate((row, features), axis=None)
  print(index)
  print(row_to_add)
  
  with open('/content/drive/MyDrive/FaceMesh shared with Mohammad/facemesh_and_features.csv', mode='a', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(row_to_add)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/FaceMesh shared with Mohammad/facemesh_and_features.csv')
#filtered_df = df[df['expression_label'].notnull()]
#df = filtered_df

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scaler
scaler = (MinMaxScaler((-1,1)))

scaled = scaler.fit_transform()

In [ ]:
X = df.drop(['expression_label', 'image_name'], axis=1) # features
y = df['expression_label'] # target

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y = lb.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)

In [ ]:
import tensorflow as tf
#from tf.losses import softmax_cross_entropy
from tensorflow import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import Adam
import keras_tuner as kt

model = Sequential()
model.add(Dense(2048, input_dim=X_train.shape[1]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))

epochs = 1000
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam() # keras default: lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0
#opt = tf.train.AdamOptimizer()
model.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer, 
    metrics=['acc'])

print("Training...")
model.fit(X_train, y_train, steps_per_epoch=100, epochs=epochs, validation_data = (X_test,y_test), batch_size=64)

#model.save('/content/drive/MyDrive/mlp.h5')

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=128)
test_loss, test_acc

# Making detections with model